In [85]:
from utilities.task_service import get_authentication_token, get_task, send_answer
from auth import apikey
import requests
# import json

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

from dotenv import load_dotenv
load_dotenv(override=True)

task_name = "knowledge"

In [86]:
# autoryzacja

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token.")
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token.


In [87]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'I will ask you a question about the exchange rate, the current population or general knowledge. Decide whether you will take your knowledge from external sources or from the knowledge of the model', 'question': 'ile orientacyjnie ludzi mieszka w Polsce?', 'database #1': 'Currency http://api.nbp.pl/en.html (use table A)', 'database #2': "Knowledge about countries https://restcountries.com/ - field 'population'"}


In [88]:
print(f"Pytanie: {response['question']}")

Pytanie: ile orientacyjnie ludzi mieszka w Polsce?


In [89]:
# wyciągnięcie populacji danego kraju
def get_population_of_country(country):
    try:
        url = f"https://restcountries.com/v3.1/name/{country}"
        response_country = requests.get(url)
        response_country.raise_for_status()  # Sprawdza, czy odpowiedź jest sukcesem (status kod 2xx)
        response_country = response_country.json()

    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
    
    population = response_country[0]['population']
    return population

In [90]:
# pobranie danych o kursach wymiany walut
try:
    url = f"http://api.nbp.pl/api/exchangerates/tables/a/"
    response_ex_rate = requests.get(url)
    response_ex_rate.raise_for_status()  # Sprawdza, czy odpowiedź jest sukcesem (status kod 2xx)
    response_ex_rate = response_ex_rate.json()

except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")
       

In [91]:
# znalezienie tematyki pytania
chat = ChatOpenAI(
    model='gpt-4'
)

question_info = chat.invoke([
    SystemMessage(f"""Przypisz pytanie użytkownika do jednej z 3 kategorii: "kursy_walut", "populacja_kraju", "wiedza_ogólna".
                   Zróć tylko tę kategorię. Na przykład:
                  ###
                  user: podaj populację Francji
                  AI: populacja_kraju

                  user: kto napisał Romeo i Julię?
                  AI: wiedza_ogólna

                  user: jaki jest dziś kurs dolara?
                  AI: kursy_walut
                  ###
                  """),
    HumanMessage(f"Pytanie: {response['question']}")
])

tematyka = question_info.content
print(tematyka)

populacja_kraju


In [92]:
# odpowiedź w zależności od typu pytania

if 'populacja_kraju' in tematyka:
    # znalezienie kraju z pytania
    country = chat.invoke([
    SystemMessage(f"""Podaj anglojęzyczną nazwę kraju, który występuje w pytaniu. Zapisz ją małymi literami.
                  ###
                  user: podaj populację Francji
                  AI: francja

                  user: podaj populację Polski
                  AI: poland
                  ###
                  """),
    HumanMessage(f"Pytanie: {response['question']}")
    ]).content

    # pobranie odpowiedzi
    answer = get_population_of_country(country)

elif 'kursy_walut' in tematyka:
    answer = chat.invoke([
    SystemMessage(f"""Odpowiedz na pytanie korzystając wyłącznie z kontekstu zamieszczonego poniżej.
                  Kontekst:###
                  {response_ex_rate}###
                  """),
    HumanMessage(f"Pytanie: {response['question']}")
    ]).content

else:
    answer = chat.invoke([
    SystemMessage(f"""Odpowiedz na pytanie."""),
    HumanMessage(f"Pytanie: {response['question']}")
    ]).content   

In [93]:
# wysłanie odpowiedzi

response = send_answer(token, answer)

if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


In [94]:
print(answer)

37950802
